# BERT Plus a Classifier

This notebook puts together a BERT model to encode then get word embeddings at a token level for the ChemE dataset (BETO). It then collates those embeddings, and sends them to a classifier that is just a default model from scikit learn, which is to be decided as to which model is best. This classifier will then classify the individual tokens based on the input vectors. It's nice to have this option, as we can remove all PAD tokens from classification I guess? 

In [1]:
import pandas as pd
import numpy as np
import torch
import random
import os
import nltk
import time
import datetime
from nltk import tokenize
from transformers import BertTokenizer

In [2]:
def extract_xy(df):
    """
    This method extracts and correctly aranges the NER training x-values (tokens)
    and y-values (BESIO labels) from a pandas dataframe containing labeled NER
    data

    Parameters:
        df (pandas DataFrame, required): Dataframe loaded via pd.read_excel() on
            a labeled NER dataset

        endings_dict (dictionary, required): Dictionary containing the indicies
            where each sentence in each line of tokens ends.

    Returns:
        two lists of identical shape. One contains all the words that were tokenized for labeling
        and the other contains all the labels of those tokenized words.
    """
    labeled = []
    columns = df.columns
    new_df = pd.DataFrame()
    all_tokens = []
    besio = []
    mol = []
    IorO = []
        
    for idx, column in enumerate(columns):
        # find every column that starts with 'name'
        if column.startswith('name'):

            # check if the entry in 'name' cell is a str
            if isinstance(df[column][0], str):
                tokens = df[columns[idx + 1]].values
                #find the index where the tokens become NaNs, and chop the token length down to that size. 
                l = 0
                for entries in tokens: 
                    if type(entries) == str:
                        l += 1
                all_tokens.append(tokens[:l])
                df[columns[idx+2]].replace(np.nan, 'O', inplace = True)
                besio.append(df[columns[idx+2]][:l].values)
                df[columns[idx+3]].replace(np.nan, '', inplace = True)
                mol.append(df[columns[idx+3]][:l].values)
                df[columns[idx+4]].replace(np.nan, '', inplace = True)
                IorO.append(df[columns[idx+4]][:l].values)

    i = 0
    label_values = []
    while i < len(besio):
        label_values.append([])
        for j in range(len(besio[i])):
            #Strip unintentional whitespace from all entries:
            besio[i][j] = besio[i][j].replace(" ", "")
            mol[i][j] = mol[i][j].replace(" ", "")
            IorO[i][j] = IorO[i][j].replace(" ", "")
            
            if besio[i][j].upper() == 'O':
                label_values[i].append(besio[i][j].upper())
            if mol[i][j].upper() == 'PRO':
                label_values[i].append(besio[i][j].upper()+'-'+mol[i][j].upper())
            if IorO[i][j].upper() == 'I' or IorO[i][j].upper() == 'O':
                #The below does not handle cases where BESIO or MOL has errors though...
                if mol[i][j].upper() != 'PRO':
                    label_values[i].append(besio[i][j].upper()+'-'+mol[i][j].upper()+'-'+IorO[i][j].upper())
                else: 
                    print("Weird. This Property is organic or inorganic? LOL")
            #PRIME OPPORTUNITY FOR ERROR HANDLING - IF ANYTHING NOT IN THE ABOVE CATS, SOMETHING IS WRONG
            
        i += 1   
    return all_tokens, label_values

In [3]:
#Ok, need another def function that takes in a list of tokens, and a list of labels, and returns them each as a list of strings
def tokenized_to_string(token_list):
    token_stringlist = []
    for paper_tokens in token_list:
        paper_string = ""
        for i in paper_tokens:
            #This is basically an 'unsplit' method lol
            paper_string += (str(i) + " ")
        token_stringlist.append(paper_string)
    return token_stringlist

In [4]:
def labeled_sheets_to_listed_tokens(directory_url):
    """This function opens a directory of labeled excel sheets from David's excel sheets and returns the tokens as a list 
    of strings fully combined on a document level. It returns a list of strings, with each string being a full document."""
    files = os.listdir(directory_url)
    token_list = []
    label_list = []
    sent_labels = []
    for file in files:
        df = pd.read_excel(directory_url+file)
        token, label = extract_xy(df)
        token_list += (tokenized_to_string(token))
        label_list += (label)
    #Now we tokenize each paper by sentences using NLTK:
    #we will also restructure labels to be ordered by sentences. 
    for i in range(len(token_list)):
        sentences = tokenize.sent_tokenize(token_list[i])
        token_list[i] = sentences
        short_term_labels = []
        for j in range(len(token_list[i])):    
            length = len(token_list[i][j].split())
            short_list = label_list[i][:length]
            short_term_labels.append(short_list)
            del(label_list[i][:length])
        sent_labels.append(short_term_labels)
    return token_list, sent_labels

In [5]:
dir_url = '/Users/Jonathan/Desktop/LabeledChemEData/Labeled_Sheets/'
list_o_tokens, list_o_labels = labeled_sheets_to_listed_tokens(dir_url)

Weird. This Property is organic or inorganic? LOL
Weird. This Property is organic or inorganic? LOL


In [6]:
def drop_empty_sentences(token_list, label_list, label_dict):
    """This function is a form of undersampling an overshadowing case, as it is dropping all examples of sentences where only O labels exist."""
    labels_list = list(label_dict.keys())
    flip = 0
    list_of_deletions = []
    del_counter = 0
    for i in range(len(labels_list)):
        if labels_list[i] == 'O':
            del(labels_list[i])
            flip = 1
        if flip == 1:
            break
    for i in range(len(token_list)):
        for j in range(len(token_list[i])):
            if any(x in label_list[i][j] for x in labels_list):
                pass
            else:
                list_of_deletions.append([i,j])
    print(list_of_deletions)
    while len(list_of_deletions) != 0:
        i,j = list_of_deletions[-1]
        del(label_list[i][j])
        del(token_list[i][j])
        del(list_of_deletions[-1])
        del_counter += 1
    print("Total deleted sentences = " + str(del_counter))
    return token_list, label_list

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

In [8]:
def max_encoded_length(token_list):
    max_len = 0
    len_list = []
    for papers in token_list:
        for sentences in papers:
            input_ids = tokenizer.encode(sentences, add_special_characters = True)
            max_len = max(max_len, len(input_ids))
            len_list.append(len(input_ids))
            
    return max_len, len_list

In [9]:
def delete_sentences(token_list, label_list, print_pop = False, max_length = 150):
    i = 0
    #lets use pop to remove these long sentences
    while i < len(token_list):
        list_of_j = []
        j = 0
        while j < len(token_list[i]):
            input_ids = tokenizer.encode(token_list[i][j], add_special_tokens = True)
            if len(input_ids) > 150:
                print("Found item length: " + str(len(input_ids)))
                list_of_j.append(j)
            j += 1
        k = len(list_of_j)-1
        #Gotta count backwards so we don't disturb the list structure
        while k > -1:
            if print_pop:
                print(token_list[i].pop(list_of_j[k]))
                print(label_list[i].pop(list_of_j[k]))
            else:
                token_list[i].pop(list_of_j[k])
                label_list[i].pop(list_of_j[k])
            k = k - 1
        i += 1 
    return token_list, label_list

In [15]:
#Not sure this is necessary if we don't train the full BERT model, but it's possible.
short_tokens, short_labels = delete_sentences(list_o_tokens, list_o_labels)

There are two label mapping dictionaries that take us from labels to numbers. One captures all of the subtleties of the BIO tagging scheme, and one deletes the BIO tags, and results in just O, MOL, PRO, and PAD/0 tags.

In [11]:
#Maybe we'll get back to this someday
label_mapping = {'O': 13, "B-MOL-O": 1, "I-MOL-O": 2, "E-MOL-O": 3,
                "S-MOL-O": 4, "B-MOL-I": 5, "I-MOL-I": 6, "E-MOL-I": 7,
                "S-MOL-I": 8, "B-PRO": 9, "I-PRO": 10, "E-PRO": 11, "S-PRO": 12,
                'PAD': 0}

In [12]:
#Long term we should absolutely consider getting rid of S, E labels
#For now, let's change O to not be the same as PAD, and see if that helps.
small_label_mapping = {'O': 3, "B-MOL-O": 1, "I-MOL-O": 1, "E-MOL-O": 1,
                "S-MOL-O": 1, "B-MOL-I": 1, "I-MOL-I": 1, "E-MOL-I": 1,
                "S-MOL-I": 1, "B-PRO": 2, "I-PRO": 2, "E-PRO": 2, "S-PRO": 2,
                'PAD': 0}


Part of the reason why we want to expand the input labels at the same level as the tokens is to make sure we have well-alligned labels and tokens for our later work when we pass both the input labels, as well as the encoded token vectors together to the final classifier.

In [20]:
def tokenize_and_align_lables(list_of_tokens, list_of_labels, encoding_dict, max_sent_length = 155):
    """"""
    input_ids = []
    attention_masks = []
    label_shapes = []
    no_pad_labels = []
    count = 0

    for abstracts, abst_labels in zip(list_of_tokens, list_of_labels):
        for sentences, sent_labels in zip(abstracts, abst_labels):
            encoded_dict = tokenizer.encode_plus(
                                        sentences,
                                        add_special_tokens = True,
                                        max_length = max_sent_length,
                                        pad_to_max_length = True,
                                        return_attention_mask = True,
                                        return_tensors = 'pt'
            )
            #Ok, now we get our labels based on encoded sizes. 
            #Make this a standalone function later instead of nesting
            #Need to start the CLS token to every label. 
            #This CLS token should be a int 0, to keep label
            #length matching consistent with the tokenized sentence
            extend_sent_labels = [0]


            #I Bet the problem with things being read in is the difference in length of 
            #however this chunks sentences versus how the labels were originally split.

            for word, label in zip(sentences, sent_labels):
                tokenized_word = tokenizer.tokenize(word)
                #Find out how many chunks each word gets broken into
                n_subwords = len(tokenized_word)
                #Extend the length of the labels to match new word length
                #Put label in brackets so it knows you want n_subwords entries
                #of label, not label times n_subwords
                extend_sent_labels.extend([label]*n_subwords)

            #In order to know just how much to bias the dataset for each label, we need to know how many of each we have. 
            no_pad_labels.extend(extend_sent_labels)

            #This handles increasing the length for padding and sep tokens
            #Go all the way to 155. Padding and  SEP should both be PAD tokens in label form
            #Because the key map will switch them to a 0. 
            extend_sent_labels.extend(['PAD']*(155-len(extend_sent_labels)))    
    #
            #Next step, we need to use the dictionary lookup
            #to replace all the values from this list to become 
            #numbers. for loops to go through the whole list. 
            for i in range(len(extend_sent_labels)):
                if extend_sent_labels[i] in encoding_dict.keys():
                    #Replace the label in extend_set_labels[i] from dict
                    extend_sent_labels[i] = encoding_dict[extend_sent_labels[i]]
            #print(extend_sent_labels)
            #Then, we make the labels list into a tensor.
            #extend_sent_labels = torch.tensor(extend_sent_labels)
            test_list = []
            test_list.append(extend_sent_labels)
            test_list = torch.tensor(test_list)
            #Build our attention mask, labels, and input ids of each item.
            label_shapes.append(test_list)
            input_ids.append(encoded_dict['input_ids'])
            attention_masks.append(encoded_dict['attention_mask'])

    print(len(label_shapes[1]))
    print(len(input_ids[1]))
    print(len(attention_masks[1]))

    #Make lists we just built into tensors
    input_ids = torch.cat(input_ids, dim = 0)
    attention_masks = torch.cat(attention_masks, dim = 0)
    labels = torch.cat(label_shapes, dim = 0)

    print("Original Sentence: ", list_of_tokens[0][0])
    print("Tokenized IDs: ", input_ids[0])
    print("Extended Labels: ", labels[0])
    
    return input_ids, attention_masks, labels

In [21]:
input_tokens, attention_mask, input_labels = tokenize_and_align_lables(short_tokens, short_labels, small_label_mapping)

1
1
1
Original Sentence:  In the interaction between gas molecules with single-walled carbon nanotube (SWCNT) we show that as a result of collisions the gas scattering contributes with an important background signal and should be considered in SWCNT-based gas sensors.
Tokenized IDs:  tensor([  101,  1999,  1996,  8290,  2090,  3806, 10737,  2007,  2309,  1011,
        17692,  6351, 28991, 28251,  2063,  1006, 25430,  2278,  3372,  1007,
         2057,  2265,  2008,  2004,  1037,  2765,  1997, 28820,  1996,  3806,
        17501, 16605,  2007,  2019,  2590,  4281,  4742,  1998,  2323,  2022,
         2641,  1999, 25430,  2278,  3372,  1011,  2241,  3806, 13907,  1012,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0

Now we need to load the tokens, labels, and the generated attention mask into a dataset and dataloader, to reduce load on the computer's memory.

In [27]:
from torch.utils.data import TensorDataset

In [24]:
dataset = TensorDataset(input_tokens, attention_mask, input_labels)

In [25]:
from torch.utils.data import DataLoader, RandomSampler

In [26]:
batch_size = 32
dataloader = DataLoader(dataset,
                        sampler = RandomSampler(dataset),
                        batch_size = batch_size
                        )

In [28]:
from transformers import BertModel, AdamW, BertConfig

In [31]:
model = BertModel.from_pretrained(
        "bert-base-uncased",
        output_attentions = False, #Whether model returns attention weights
        output_hidden_states = False, #Whether model outputs all hidden states                                    
        )

In [33]:
model.cpu()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

Ok, we have the model set up. We can do one of two things: We can build the vectors without fine-tuning the BERT model. That would be simplest honestly. We can do that first, and then once we get a classifier working we can work to fine tune that to a greater extent.

In [37]:
#We do this without the dataloader, only inputting the Input IDs.
#This is NOT FAST. Start 4:05ish. It takes at least 10 minutes to run through.
with torch.no_grad():
    last_hidden_states = model(input_tokens)

In [52]:
len(last_hidden_states[0][1][1])

768

Ok, each of these elements in last_hidden_states have a word, and a vector of size 1, 768. 

I'm not sure what the different outputs mean yet. Let me read Jay Alammar's blog thing:
Here's the structure. We want back out element 0 in the tuple of the model's output. These represent our word embeddings. Those elements have a shape of 1268 (ie, number of sentences input), by 155 (length of the sentences), by 768 (the length of the vector embedding for each word). 
So, in order to characterize each word, we'll want to include the embedding for each word as an input into the final neural network. 

In [53]:
embedding_vectors = last_hidden_states[0]

Ok, now that we have the embedding vectors for each specific word, it's time to put them into the new classifier neural network. 

Ok, to do that, our first step is to make sure that we have clean alignment and the same shape between everything. 
I'm actually not sure how to handle the full vector, considering that the hidden states is going to have a value of 768 for each word, and the labels are 1 for each word. I'm sure that's taken in somewhere. 

In [54]:
from sklearn.linear_model import LogisticRegression

LogisticRegression module uses an input of n_samples, n_features for the x values position, and n_samples for the labels position. So, we need to remove some of the structure, but not all of it. We basically just need to read through the full list of embedding vectors, reading each sentence at a time, and making a list of lists. Not bad!

Once we have things like that, that's not such a bad thing. Other models like randomforest also need this type of architecture. If we end up building our own NNs to take into consideration sentence structures also, then we can just take the raw BERT output and use that. 

In [56]:
len(embedding_vectors[0])

155

In [57]:
len(embedding_vectors[0][1])

768

In [85]:
numpy_embeddings = embedding_vectors.numpy()

In [86]:
#Ok, could improve this by removing the PAD tokens when we get here. 
#Probably the best thing we can do is drop the first one, and then drop the 
#Tokens that are shown as the attention mask, and then also delete the token 
#Where the attention mask of i plus attention mask i+1 == 1, as that's the SEP Token
reshaped_embeddings = []
for i in range(len(numpy_embeddings)):
    for j in range(len(numpy_embeddings[i])):
        reshaped_embeddings.append(numpy_embeddings[i][j])

In [113]:
len(reshaped_embeddings)

197160

In [114]:
len(reshaped_embeddings[154])

768

In [115]:
type(reshaped_embeddings[1])

numpy.ndarray

In [72]:
len(input_labels[0])

155

Ok, now we need to reshape the labels, and send the to a numpy array, instead of a torch tensor. 

In [90]:
numpy_labels = input_labels.numpy()

In [91]:
reshaped_labels = []
for i in range(len(numpy_labels)):
    for j in range(len(numpy_labels[i])):
        reshaped_labels.append(numpy_labels[i][j])

In [92]:
len(reshaped_labels)

197160

Ok, now everything is SET! We have our arrays in the right shapes, and they should be ok to input into a lmfit model. That is a job for tomorrow.

In [93]:
numpy_attention = attention_mask.numpy()

In [94]:
#Next step, need to remove all padding from all the tokens
reshaped_attention_mask = []
for i in range(len(numpy_attention)):
    for j in range(len(numpy_attention[i])):
        reshaped_attention_mask.append(numpy_attention[i][j])

In [95]:
type(reshaped_attention_mask[1])

numpy.int64

In [100]:
reshaped_attention_mask[:158]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1]

Ok, our next step is to drop every place in all three matrices where the attention mask equals zero. 

In [101]:
list_of_pads = []
for i in range(len(reshaped_labels)):
    #Now we start looking at the atten mask and see where it's 0.
    if reshaped_attention_mask[i] == 0:
        list_of_pads.append(i)      

In [107]:
len(list_of_pads)

147257

In [116]:
#Ok, now that we have a positional list of all the indices of pads, we delete them.
#This doesn't work unfortunately. We'll have to do it ourselves.
# no_pad_labels = np.delete(reshaped_labels, list_of_pads)
# no_pad_tokens = np.delete(reshaped_embeddings, list_of_pads, axis = 0)

In [148]:
#First, we need to sort the list of pads by descending, so we can start at the end of the list.
desc_list_pads = sorted(list_of_pads, reverse = True)

In [149]:
desc_list_pads[:100]

[197159,
 197158,
 197157,
 197156,
 197155,
 197154,
 197153,
 197152,
 197151,
 197150,
 197149,
 197148,
 197147,
 197146,
 197145,
 197144,
 197143,
 197142,
 197141,
 197140,
 197139,
 197138,
 197137,
 197136,
 197135,
 197134,
 197133,
 197132,
 197131,
 197130,
 197129,
 197128,
 197127,
 197126,
 197125,
 197124,
 197123,
 197122,
 197121,
 197120,
 197119,
 197118,
 197117,
 197116,
 197115,
 197114,
 197113,
 197112,
 197111,
 197110,
 197109,
 197108,
 197107,
 197106,
 197105,
 197104,
 197103,
 197102,
 197101,
 197100,
 197099,
 197098,
 197097,
 197096,
 197095,
 197094,
 197093,
 197092,
 197091,
 197090,
 197089,
 197088,
 197087,
 197086,
 197085,
 197084,
 197083,
 197082,
 197081,
 197080,
 197079,
 197078,
 197077,
 197076,
 197075,
 197074,
 197073,
 197072,
 197071,
 197070,
 197069,
 197068,
 197067,
 197066,
 197065,
 197064,
 197063,
 197062,
 197061,
 197060]

In [184]:
padded_percentage_list = [0,0,0,0]
for i in range(len(reshaped_labels)):
    padded_percentage_list[reshaped_labels[i]] += 1

In [185]:
padded_percentage_list

[173810, 2904, 987, 19459]

In [186]:
173810/(173810+2904+987+19459)

0.881568269425847

In [157]:
#Cool, now we just iterate through the list and delete every index found inthis list.
no_pad_labels = reshaped_labels.copy()
no_pad_tokens = reshaped_embeddings.copy()
#Now we delete every index from the labels and tokens!
for i in desc_list_pads:
    #print(i)
    #print(no_pad_labels[i])
    del(no_pad_labels[i])
    del(no_pad_tokens[i])

In [169]:
reshaped_labels[40:60]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [170]:
no_pad_labels[40:60]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3]

In [171]:
reshaped_attention_mask[40:60]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [167]:
list_of_pads[:15]

[51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65]

In [164]:
len(reshaped_labels)

197160

In [165]:
#Needs a train_test split eventually.
len(no_pad_labels)

49903

In [144]:
len(no_pad_tokens)

49903

In [145]:
len(no_pad_tokens[1])

768

In [179]:
percentage_list = [0,0,0,0]
for i in range(len(no_pad_labels)):
    percentage_list[no_pad_labels[i]] += 1

In [180]:
percentage_list

[26553, 2904, 987, 19459]

In [189]:
2904/(sum(percentage_list)-26553)

0.12436830835117774

In [195]:
current_len = len(no_pad_labels)-1
while -1 < current_len:
    if no_pad_labels[current_len] == 0:
        del(no_pad_labels[current_len])
        del(no_pad_tokens[current_len])
    current_len -= 1      

In [205]:
len(no_pad_tokens)

23350

In [208]:
len(no_pad_tokens[1])

768

In [210]:
type(no_pad_tokens[1])

numpy.ndarray

In [196]:
len(no_pad_labels)

23350

In [197]:
final_percentage_list = [0,0,0,0]
for i in range(len(no_pad_labels)):
    final_percentage_list[no_pad_labels[i]] += 1

In [198]:
final_percentage_list

[0, 2904, 987, 19459]

In [213]:
final_percentage_list[3]/(sum(final_percentage_list))

0.8333618843683084

In [121]:
#Alright, now that we've removed all the padding components, let's put in a model!
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [199]:
#Split on a 75-25 to start, we can modify later. 
train_tokens, test_tokens, train_labels, test_labels  = train_test_split(no_pad_tokens, no_pad_labels)

In [200]:
len(train_tokens)

17512

In [211]:
lr_clf = LogisticRegression()
lr_clf.fit(train_tokens, train_labels)

C:\Users\Jonathan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [212]:
lr_clf.score(test_tokens, test_labels)

0.8273381294964028

In [222]:
predictions_for_test = lr_clf.predict(test_tokens)

In [223]:
guess_list = [0,0,0,0]
for i in range(len(predictions_for_test)):
    guess_list[predictions_for_test[i]] += 1

In [224]:
guess_list

[0, 127, 10, 5701]

In [226]:
percentage_list = [0,0,0,0]
for i in range(len(test_labels)):
    percentage_list[test_labels[i]] += 1

In [227]:
percentage_list

[0, 742, 231, 4865]

In [228]:
percentage_list[3]/sum(percentage_list)

0.8333333333333334

In [215]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [217]:
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_tokens, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

C:\Users\Jonathan\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469

best parameters:  {'C': 0.0001}
best scrores:  0.8333714024668799


Ok, now let's try to deal with introducing weights to the model first, using a built in method of CRF: class_weight : dict or 'balanced', optional (default=None)

In [229]:
weighted_LR_CLF = LogisticRegression(class_weight = 'balanced')

In [230]:
weighted_LR_CLF.fit(train_tokens, train_labels)

C:\Users\Jonathan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [231]:
weighted_LR_CLF.score(test_tokens, test_labels)

0.7423775265501884

In [232]:
weighted_predictions = weighted_LR_CLF.predict(test_tokens)

In [233]:
weighted_guess_list = [0,0,0,0]
for i in range(len(weighted_predictions)):
    weighted_guess_list[weighted_predictions[i]] += 1

In [234]:
weighted_guess_list

[0, 706, 429, 4703]

In [235]:
percentage_list

[0, 742, 231, 4865]

In [236]:
from sklearn.metrics import confusion_matrix

In [238]:
conf_mat = confusion_matrix(test_labels, weighted_predictions)

In [242]:
conf_mat

array([[ 220,   61,  461],
       [  23,   40,  168],
       [ 463,  328, 4074]], dtype=int64)

In [239]:
import pandas as pd

In [251]:
confusion_mat = pd.DataFrame(conf_mat, columns = ["Actual 1", "Actual 2", "Actual 3"], index = ["Guess 1", "Guess 2", "Guess 3"])

In [252]:
confusion_mat

,Actual 1,Actual 2,Actual 3
Guess 1,220,61,461
Guess 2,23,40,168
Guess 3,463,328,4074


In [254]:
params = {'C': np.linspace(0.0001, 100, 20)}
weight_grid_search = GridSearchCV(LogisticRegression(class_weight = 'balanced'), params)
weight_grid_search.fit(train_tokens, train_labels)

print('best parameters: ', weight_grid_search.best_params_)
print('best scrores: ', weight_grid_search.best_score_)

C:\Users\Jonathan\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469

best parameters:  {'C': 0.0001}
best scrores:  0.8318296025582458


In [255]:
#Started at 4:22. Done at 5:10. Best param 0.0001, score 0.831829

In [295]:
optimized_weighted_LR_CLF = LogisticRegression(class_weight = 'balanced', C = 0.01)

In [296]:
optimized_weighted_LR_CLF.fit(train_tokens, train_labels)

C:\Users\Jonathan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=0.01, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [297]:
opt_weight_predicts = optimized_weighted_LR_CLF.predict(test_tokens)

In [298]:
opt_weighted_guess_list = [0,0,0,0]
for i in range(len(opt_weight_predicts)):
    opt_weighted_guess_list[opt_weight_predicts[i]] += 1

In [299]:
opt_weighted_guess_list

[0, 570, 229, 5039]

In [300]:
opt_weight_conf_mat = confusion_matrix(opt_weight_predicts, test_labels)
opt_weight_confusion_mat = pd.DataFrame(opt_weight_conf_mat, columns = ["Actual 1", "Actual 2", "Actual 3"], index = ["Guess 1", "Guess 2", "Guess 3"])

In [301]:
opt_weight_confusion_mat

,Actual 1,Actual 2,Actual 3
Guess 1,188,18,364
Guess 2,32,26,171
Guess 3,522,187,4330


In [302]:
optimized_weighted_LR_CLF.score(test_tokens, test_labels)

0.7783487495717711